### General
Based on https://www.kaggle.com/code/royalacecat/pesudo-shake-shake-up
### Use case
This notebook is used to load the pretrained checkpoints as datasets and execute it on Kaggle.

### Submission only

In [1]:
import torch
import sys
import gc
import os
import glob
import yaml
import pandas as pd
import numpy as np
import torch.nn as n


In [2]:
!pip install Lightning -q
!pip install segmentation_models_pytorch -q

ERROR: Could not find a version that satisfies the requirement Lightning (from versions: none)
ERROR: No matching distribution found for Lightning
ERROR: Could not find a version that satisfies the requirement segmentation_models_pytorch (from versions: none)
ERROR: No matching distribution found for segmentation_models_pytorch


In [3]:
import pytorch_lightning as L
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np
import os
import pandas as pd
sys.path.append("../input/pretrained-models-pytorch")
sys.path.append("../input/efficientnet-pytorch")
sys.path.append("/kaggle/input/smp-github/segmentation_models.pytorch-master")
sys.path.append("/kaggle/input/timm-pretrained-resnest/resnest/")
sys.path.append("/kaggle/input/smp-github/segmentation_models.pytorch-master")
import segmentation_models_pytorch as smp
import torchvision.transforms as T
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.optim import AdamW
from torchmetrics.functional import dice
from torch.utils.data import Dataset, DataLoader
import warnings
import yaml

import warnings
warnings.filterwarnings("ignore")
torch.set_float32_matmul_precision("medium")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
seg_models = {
    "Unet": smp.Unet
}

In [5]:
%%writefile config_resnest101_Unet_pretrained.yaml

model:
    seg_model: "Unet"
    encoder_name: "timm-resnest101e"
    loss_smooth: 1.0
    image_size: 384

Writing config_resnest101_Unet_pretrained.yaml


In [6]:
with open("config_resnest101_Unet_pretrained.yaml", "r") as file_obj:
    config_resnest101_Unet_pretrained = yaml.safe_load(file_obj)

L.seed_everything(20)
gc.enable()

In [7]:
class ContrailsDataset(torch.utils.data.Dataset):
    def __init__(self, df, image_size=256, train=True):
        
        self.df = df
        self.trn = train
        self.df_idx: pd.DataFrame = pd.DataFrame({'idx': os.listdir(f'/kaggle/input/google-research-identify-contrails-reduce-global-warming/test')})
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.image_size = image_size
        if image_size != 256:
            self.resize_image = T.transforms.Resize(image_size)
    
    def read_record(self, directory):
        record_data = {}
        for x in [
            "band_11", 
            "band_14", 
            "band_15"
        ]:

            record_data[x] = np.load(os.path.join(directory, x + ".npy"))

        return record_data

    def normalize_range(self, data, bounds):
        """Maps data to the range [0, 1]."""
        return (data - bounds[0]) / (bounds[1] - bounds[0])
    
    def get_false_color(self, record_data):
        _T11_BOUNDS = (243, 303)
        _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
        _TDIFF_BOUNDS = (-4, 2)
        
        N_TIMES_BEFORE = 4

        r = self.normalize_range(record_data["band_15"] - record_data["band_14"], _TDIFF_BOUNDS)
        g = self.normalize_range(record_data["band_14"] - record_data["band_11"], _CLOUD_TOP_TDIFF_BOUNDS)
        b = self.normalize_range(record_data["band_14"], _T11_BOUNDS)
        false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
        img = false_color[..., N_TIMES_BEFORE]

        return img
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        con_path = row.path
        data = self.read_record(con_path)    
        
        img = self.get_false_color(data)
        
        img = torch.tensor(np.reshape(img, (256, 256, 3))).to(torch.float32).permute(2, 0, 1)
        
        if self.image_size != 256:
            img = self.resize_image(img)
        
        img = self.normalize_image(img)
        
        image_id = int(self.df_idx.iloc[index]['idx'])
            
        return img.float(), torch.tensor(image_id)
    
    def __len__(self):
        return len(self.df)

In [8]:
#mine
data_root_folder = '/kaggle/input/google-research-identify-contrails-reduce-global-warming'
data_path_train = "/kaggle/input/google-research-identify-contrails-reduce-global-warming/train/"
data_path_valid = "/kaggle/input/google-research-identify-contrails-reduce-global-warming/validation/"
data_path_test = "/kaggle/input/google-research-identify-contrails-reduce-global-warming/test/"
ckpt_path_pre = "/kaggle/input/pretrained-models-for-5-timesteps/"
ckpt_path_top = "/kaggle/input/top-model-state-dict/top_model_state_dict.pt"


sys.path.append(data_path_train)
sys.path.append(data_path_valid)
sys.path.append(data_path_test)
sys.path.append(ckpt_path_pre)
sys.path.append(ckpt_path_top)


checkpoint_paths = {
    "-1": ckpt_path_pre + "model_at_timestep_mask_plus_delta_-1.ckpt",
    "0": ckpt_path_pre + "model_at_timestep_mask_plus_delta_0.ckpt",
    "1": ckpt_path_pre + "model_at_timestep_mask_plus_delta_1.ckpt",
    "2": ckpt_path_pre + "model_at_timestep_mask_plus_delta_2.ckpt",
    "3": ckpt_path_pre + "model_at_timestep_mask_plus_delta_3.ckpt"   
}

In [9]:
batch_size = 32
num_workers = 2
THR = 0.45
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
submission = pd.read_csv(os.path.join(data_root_folder, 'sample_submission.csv'), index_col='record_id')

filenames = os.listdir(data_path_test)
test_df = pd.DataFrame(filenames, columns=['record_id'])
test_df['path'] = data_path_test + test_df['record_id'].astype(str)

In [10]:
#mine
class ContrailsDatasetMixed(Dataset):
    def __init__(self, split="test", mode="single", delta_t = 0, df = test_df):
        self.df = df
        self.df_idx: pd.DataFrame = pd.DataFrame({'idx': os.listdir(f'/kaggle/input/google-research-identify-contrails-reduce-global-warming/test')})
        
        self.delta_t = delta_t
        self.split = split
        self.mode = mode
        self.path =  {"train" :data_path_train, "valid": data_path_valid, "test": data_path_test}[self.split]
        self.examples = os.listdir(self.path)
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    
    def read_record(self, directory):
        record_data = {}
        for x in [
            "band_11", 
            "band_14", 
            "band_15"
        ]:
            record_data[x] = np.load(os.path.join(directory, x + ".npy")) #?

        return record_data

    def normalize_range(self, data, bounds):
        """Maps data to the range [0, 1]."""
        return (data - bounds[0]) / (bounds[1] - bounds[0])
    
    def get_false_color(self, record_data):
        _T11_BOUNDS = (243, 303)
        _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
        _TDIFF_BOUNDS = (-4, 2)
        r= self.normalize_range(record_data["band_15"] - record_data["band_14"], _TDIFF_BOUNDS)
        g = self.normalize_range(record_data["band_14"] - record_data["band_11"], _CLOUD_TOP_TDIFF_BOUNDS)
        b = self.normalize_range(record_data["band_14"], _T11_BOUNDS)
        false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
        if self.mode == "single":
            t_null = 4
            return false_color[..., t_null+self.delta_t]
        else:
            return false_color
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        con_path = row.path
        data = self.read_record(con_path)  
        img = self.get_false_color(data)
        

        if self.mode == "single":
            img = torch.tensor(np.reshape(img, (256, 256, 3, -1))).to(torch.float32).permute(3, 2, 0, 1).squeeze()
        else:
            img = torch.tensor(np.reshape(img, (256, 256, 3, -1))).to(torch.float32).permute(3, 2, 0, 1)

        img = self.normalize_image(img)
        image_id = int(self.df_idx.iloc[index]['idx'])


        if self.split in ["train", "validation"]:
            label = np.load(os.path.join(con_path, "human_pixel_masks.npy")).squeeze()
            label = torch.Tensor(label).to(torch.int64)
            return img.float(), label.float()

        return img.float(), torch.tensor(image_id)
    
    def __len__(self):
        return len(self.examples)

In [11]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s

In [12]:
#mine
class LightningModule_pre(L.LightningModule):
    def __init__(self):
        super().__init__()
        config = config_resnest101_Unet_pretrained["model"] #passing config as parameter produces error when loading checkpoints, even from already initialized model
        self.config = config
        self.model = model = seg_models[config["seg_model"]](
            encoder_name=config["encoder_name"],
            encoder_weights=None,
            in_channels=3,
            classes=1,
            activation=None,
        )

    def forward(self, batch):
        imgs = batch
        preds = self.model(imgs)
        return preds

In [13]:
#mine
class LightningModule_top(L.LightningModule):
    
    def __init__(self):
        super().__init__()
        config = config_resnest101_Unet_pretrained["model"] #passing config as parameter produces error when loading checkpoints, even from already initialized model
        self.config = config
        self.model = model = seg_models[config["seg_model"]](
            encoder_name=config["encoder_name"],
            encoder_weights=None,
            in_channels=5,
            classes=1,
            activation=None,
        )

    def forward(self, batch):
        imgs = batch
        preds = self.model(imgs)
        return preds

In [14]:
#mine
# test_ds = ContrailsDatasetMixed(split="test", mode="multi")
 
# test_dl = DataLoader(test_ds, batch_size=batch_size, num_workers = num_workers)

In [15]:
# test_ds = ContrailsDataset(
#         test_df,
#         config["model"]["image_size"],
#         train = False
#     )
 
# test_dl = DataLoader(test_ds, batch_size=batch_size, num_workers = num_workers)

In [16]:
checkpoint_paths

{'-1': '/kaggle/input/pretrained-models-for-5-timesteps/model_at_timestep_mask_plus_delta_-1.ckpt',
 '0': '/kaggle/input/pretrained-models-for-5-timesteps/model_at_timestep_mask_plus_delta_0.ckpt',
 '1': '/kaggle/input/pretrained-models-for-5-timesteps/model_at_timestep_mask_plus_delta_1.ckpt',
 '2': '/kaggle/input/pretrained-models-for-5-timesteps/model_at_timestep_mask_plus_delta_2.ckpt',
 '3': '/kaggle/input/pretrained-models-for-5-timesteps/model_at_timestep_mask_plus_delta_3.ckpt'}

In [17]:
#-----------------------------------------------mine-----------------------------------------------
config = config_resnest101_Unet_pretrained

gc.enable()

all_preds = {}

for k in checkpoint_paths:
    v = checkpoint_paths[k]
    test_ds = ContrailsDatasetMixed(split="test", mode="single", delta_t = int(k), df = test_df)
    test_dl = DataLoader(test_ds, batch_size=batch_size, num_workers = num_workers)
    model = LightningModule_pre().load_from_checkpoint(str(v))
    model.to(device)
    model.eval()

    model_preds = {}
    
    for _, data in enumerate(test_dl):
        images, image_id = data
    
        images = images.to(device)
        
        with torch.no_grad():
            predicted_mask = model(images[:, :, :, :])
        if config["model"]["image_size"] != 256:
            predicted_mask = torch.nn.functional.interpolate(predicted_mask, size=256, mode='bilinear')
                
        for img_num in range(0, images.shape[0]):
            current_mask = predicted_mask[img_num, :, :, :]
            current_image_id = image_id[img_num].item()
            model_preds[current_image_id] = current_mask
    all_preds[f"{k}"] = model_preds
    
    del model    
    torch.cuda.empty_cache()
    gc.collect() 

In [18]:
#-----------------------------------------------mine-----------------------------------------------

top_model = LightningModule_top()
top_model.load_state_dict(torch.load(ckpt_path_top)["model_state_dict"])
top_model.to(device)
top_model.eval()

top_model_mask = {}

for index in submission.index.tolist():
    masks = []
    for pre_model in all_preds.keys():
        pred_mask = all_preds[pre_model][index]
        masks.append(pred_mask)
    combined_masks = torch.stack(masks, dim = 1).to(device) #dim 0 is batch #todo:make dataset of outputs
    top_model_mask = top_model.forward(combined_masks).squeeze()
    top_model_mask = torch.sigmoid(top_model_mask).cpu().detach().numpy()
    predicted_mask_with_threshold = np.zeros((256, 256))
    predicted_mask_with_threshold[top_model_mask[ :, :] < 0.45] = 0
    predicted_mask_with_threshold[top_model_mask[ :, :] > 0.45] = 1
    submission.loc[int(index), 'encoded_pixels'] = list_to_string(rle_encode(predicted_mask_with_threshold))
    
    
del top_model    
torch.cuda.empty_cache()
gc.collect() 

8

In [19]:
submission.to_csv('submission.csv')